# Modeling

### Getting encoded data

Since we have to do feature selection again we have to do one hot encoding again

In [1]:
import pandas as pd
import numpy as np

In [2]:
cleaned_df = pd.read_csv("cleaned_df.csv") 

In [3]:
list(cleaned_df.columns)

['Unnamed: 0',
 'ARM',
 'HR1_10SCALE',
 'HR2_1',
 'HR2_2',
 'HR2_3',
 'HR2_4',
 'HR2_5',
 'HR3',
 'HR4',
 'HR5',
 'HR6',
 'HR7_1',
 'HR7_2',
 'HR7_3',
 'HR7_4',
 'HR8',
 'HR11',
 'HR12_1',
 'HR12_2',
 'HR12_3',
 'HR12_4',
 'HR12_5',
 'HR13',
 'HR14',
 'COVID_HR1',
 'COVID_HR2',
 'COVID_HR3',
 'STIMULUS2',
 'STIMULUS3',
 'STIMULUS4',
 'STIMULUS5',
 'STIMULUS6',
 'POLICY1',
 'POLICY2',
 'POLICY3',
 'POLICY4',
 'PERSONAL2',
 'PERSONAL3',
 'PERSONAL4',
 'WORRY_1',
 'WORRY_2',
 'WORRY_3',
 'WORRY_4',
 'WORRY_5',
 'WORRY_6',
 'WORRY_7',
 'WORRY_8',
 'MEDIA1',
 'MEDIA2',
 'MEDIA5',
 'MEDIA6_1',
 'MEDIA6_2',
 'MEDIA6_3',
 'MEDIA6_4',
 'MEDIA6_5',
 'MEDIA6_6',
 'MEDIA6_7',
 'MEDIA7',
 'MEDIA9',
 'MEDIA10',
 'MEDIA11',
 'MEDIA12',
 'MEDIA13',
 'MEDIA14',
 'MEDIA15',
 'MEDIA17',
 'MH1',
 'MH2',
 'CHILDCARE1',
 'CHILDCARE2',
 'CHILDCARE3',
 'ELDERLYCARE1',
 'IDEOLOGY1',
 'IDEOLOGY2',
 'IDEOLOGY3',
 'IDEOLOGY4',
 'IDEOLOGY5',
 'IDEOLOGY6',
 'IDEOLOGY7',
 'IDEOLOGY8',
 'CULTURE1_1',
 'CULTURE1_2',
 

In [4]:
cleaned_df = cleaned_df.drop(["COVID_HR1","COVID_HR2","IDEOLOGY4"],axis=1)
missing = ["HR11","IDEOLOGY7"]
for i in missing:
    mode = cleaned_df[i].mode()
    print(mode[0])
    fixed = []
    for value in cleaned_df[i]:
        if value == 'NA' or value != value:
            fixed.append(mode[0])
        else:
            fixed.append(value)
    cleaned_df[i] = fixed

B
C


In [5]:
for (columnName, columnData) in cleaned_df.iteritems(): 
    missingCount = (columnData == 'NA').sum() 
    missingCount += columnData.isnull().sum() 
    percentage = (missingCount/len(cleaned_df))*100 
    #print(percentage)
    if missingCount > 0:        
        print("Missing Values in column : ", columnName, ", missing rows: ", 
            missingCount, ", missing percentage: ",  percentage, '%') 
        if percentage > 50.0:
            too_many_missing_value_columns.append(columnName) 

### Feature selection
Feature we are going to use here is the ordinal IDEOLOGY5. IDEOLOGY5 is "If you had to describe your ideology along a Left-Right political spectrum, which of the following would come closest to how you would describe yourself?"

In [6]:
cleaned_df["HR6"].unique()

array([3, 2, 0, 4, 1], dtype=int64)

In [8]:
tmp_df = cleaned_df.copy()
nominal_val = cleaned_df.select_dtypes(include=['object']).columns
for index, values in tmp_df.iteritems():
    if index in nominal_val:
        unique = pd.unique(values)
        dic = []
        for i in values:
            dic.append(np.where(unique == i)[0][0])
        tmp_df[index] = dic

In [9]:
tmp_df.columns

Index(['Unnamed: 0', 'ARM', 'HR1_10SCALE', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4',
       'HR2_5', 'HR3', 'HR4',
       ...
       'RUMORS1_USARMY_BROUGHT', 'RUMORS1_BAT_TO_HUMANS',
       'RUMORS1_BAT_ANIMAL_HUMANS', 'RUMORS1_WET_MARKET', 'RUMORS1_OTHER',
       'DEMOGRAPHIC1_EMPLOYED', 'DEMOGRAPHIC1_UNEMPLOYED',
       'DEMOGRAPHIC1_RETIRED', 'DEMOGRAPHIC1_STUDENTS', 'DEMOGRAPHIC1_OTHER'],
      dtype='object', length=216)

IDEOLOGY5 contains negative values. We have to convert those to positive

In [12]:
ide5 = list(tmp_df["IDEOLOGY5"])
vals = [-2,-1,0,1,2]
test = []
for i in ide5:
    test.append(vals.index(i))
tmp_df["IDEOLOGY5"] = test

In [13]:
from sklearn.model_selection import train_test_split
X = tmp_df.drop(['HR6'],axis=1) #obamacare
y = tmp_df['HR6']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

In [14]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(x_train,y_train)
x_train_fs = fs.transform(x_train)
x_test_fs = fs.transform(x_test)

In [17]:
from matplotlib import pyplot

selected_features = []
for i in range(len(fs.scores_)):
    #print('Feature %s: %f' % (tmp_df.columns[i], fs.scores_[i]))
    if fs.scores_[i] > 150:
        print('Feature %s: %f' % (X.columns[i], fs.scores_[i]))
        selected_features.append(X.columns[i])
selected_features.remove("Unnamed: 0")
selected_features

Feature Unnamed: 0: 1118.174156
Feature HR2_1: 186.493677
Feature HR2_2: 229.000817
Feature HR4: 226.407734
Feature HR5: 173.098587
Feature PERSONAL3: 159.764489
Feature MEDIA10: 175.402413
Feature MEDIA12: 175.526229
Feature DEMOGRAPHIC5: 6782.521443
Feature age: 454.866102


['HR2_1',
 'HR2_2',
 'HR4',
 'HR5',
 'PERSONAL3',
 'MEDIA10',
 'MEDIA12',
 'DEMOGRAPHIC5',
 'age']

Selected Features are 'HR6', 'MEDIA12', 'IDEOLOGY6', 'DEMOGRAPHIC4', 'DEMOGRAPHIC5' DEMOGRAPHIC 4 is nominal so it will have to be encoded.

### One Hot

In [18]:
selected_df = cleaned_df[selected_features].copy()
selected_df['HR6'] = cleaned_df['HR6'].copy()
selected_df

,HR2_1,HR2_2,HR4,HR5,PERSONAL3,MEDIA10,MEDIA12,DEMOGRAPHIC5,age,HR6
0,3,1,2,3,3,3,1,46825,24,3
1,3,3,2,3,2,3,1,81650,23,2
2,0,0,0,0,0,0,0,31027,27,0
3,3,3,1,2,0,0,2,60609,51,2
4,3,0,4,1,4,4,1,39466,37,4
...,...,...,...,...,...,...,...,...,...,...
1206,1,2,4,4,1,0,0,43701,22,3
1207,5,5,4,4,4,1,1,34475,24,4
1208,4,4,3,3,1,3,0,6152,20,2
1209,5,5,4,4,2,3,0,21740,48,4


In [19]:
nominal_df = selected_df[selected_df.select_dtypes(include=['object']).columns]
nominal_df

,DEMOGRAPHIC5
0,46825
1,81650
2,31027
3,60609
4,39466
...,...
1206,43701
1207,34475
1208,6152
1209,21740


In [20]:
nn_df = pd.DataFrame()
for i in nominal_df.columns:
    unique = pd.unique(selected_df[i].copy())
    count = 0
    for j in unique:
        name = i + "_" + str(j)
        tmp = []
        for k in nominal_df[i]:
            if k == j:
                tmp.append(1)
            else:
                tmp.append(0)
        count +=1
        nn_df[name] = tmp
nn_df

<ipython-input-20-3de23106784c>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  nn_df[name] = tmp


,DEMOGRAPHIC5_46825,DEMOGRAPHIC5_81650,DEMOGRAPHIC5_31027,DEMOGRAPHIC5_60609,DEMOGRAPHIC5_39466,DEMOGRAPHIC5_60644,DEMOGRAPHIC5_76301,DEMOGRAPHIC5_98409,DEMOGRAPHIC5_61614,DEMOGRAPHIC5_37388,...,DEMOGRAPHIC5_12225,DEMOGRAPHIC5_31323,DEMOGRAPHIC5_90005,DEMOGRAPHIC5_18051,DEMOGRAPHIC5_25271,DEMOGRAPHIC5_43701,DEMOGRAPHIC5_34475,DEMOGRAPHIC5_6152,DEMOGRAPHIC5_21740,DEMOGRAPHIC5_40353
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
selected_df = selected_df.drop(nominal_df.columns,axis=1)
for index, value in nn_df.iteritems():
    selected_df[index] = value

<ipython-input-21-02f9fd1118ec>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  selected_df[index] = value


In [24]:
selected_df.to_csv("ideology5_encoded.csv")
selected_df

,HR2_1,HR2_2,HR4,HR5,PERSONAL3,MEDIA10,MEDIA12,age,HR6,DEMOGRAPHIC5_46825,...,DEMOGRAPHIC5_12225,DEMOGRAPHIC5_31323,DEMOGRAPHIC5_90005,DEMOGRAPHIC5_18051,DEMOGRAPHIC5_25271,DEMOGRAPHIC5_43701,DEMOGRAPHIC5_34475,DEMOGRAPHIC5_6152,DEMOGRAPHIC5_21740,DEMOGRAPHIC5_40353
0,3,1,2,3,3,3,1,24,3,1,...,0,0,0,0,0,0,0,0,0,0
1,3,3,2,3,2,3,1,23,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,27,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,1,2,0,0,2,51,2,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,4,1,4,4,1,37,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,1,2,4,4,1,0,0,22,3,0,...,0,0,0,0,0,1,0,0,0,0
1207,5,5,4,4,4,1,1,24,4,0,...,0,0,0,0,0,0,1,0,0,0
1208,4,4,3,3,1,3,0,20,2,0,...,0,0,0,0,0,0,0,1,0,0
1209,5,5,4,4,2,3,0,48,4,0,...,0,0,0,0,0,0,0,0,1,0


### Modeling

In [22]:
X = selected_df.drop('HR6',axis=1)
y = selected_df['HR6']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

In [23]:
import tensorflow as tf

ANN can't take negative values so we have to convert it

In [25]:
y.unique()

array([3, 2, 0, 4, 1], dtype=int64)

In [26]:
tmp_y_train = []
values = [0,1,2,3,4]
encod = [[0,0,0,0],
        [1,0,0,0],
        [1,1,0,0],
        [1,1,1,0],
        [1,1,1,1]]
for i in y_train:
    tmp_y_train.append(encod[values.index(i)])
tmp_y_test = []
for i in y_test:
    tmp_y_test.append(encod[values.index(i)])
tmp_y_train = np.array(tmp_y_train)
tmp_y_test = np.array(tmp_y_test)

In [51]:
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Dense(units = 1, activation = 'relu', input_shape = (None, 949)))

In [57]:
model1.add(tf.keras.layers.Dropout(0.2))
model1.add(tf.keras.layers.Dense(units = 4, activation = 'sigmoid'))
model1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
model1.fit(x_train, tmp_y_train, epochs =5)

Epoch 1/5
29/29 [==============================] - 1s 1ms/step - loss: 3.3643 - categorical_accuracy: 0.0286
Epoch 2/5
29/29 [==============================] - 0s 1ms/step - loss: 3.2920 - categorical_accuracy: 0.0330
Epoch 3/5
29/29 [==============================] - 0s 1ms/step - loss: 3.2378 - categorical_accuracy: 0.1112
Epoch 4/5
29/29 [==============================] - 0s 1ms/step - loss: 3.1880 - categorical_accuracy: 0.1465
Epoch 5/5
29/29 [==============================] - 0s 2ms/step - loss: 3.1707 - categorical_accuracy: 0.1707


In [60]:
test_loss, test_accuracy = model1.evaluate(x_test, tmp_y_test)
print(test_accuracy)
print(x_test.columns)
idk = model1.predict(x_test)
print(idk)
print(y_test)
for i in range(len(idk)):
    print(np.sum(tmp_y_test[i]),np.round(np.sum(idk[i])))

10/10 [==============================] - 0s 1ms/step - loss: 3.0619 - categorical_accuracy: 0.0000e+00
0.0
Index(['HR2_1', 'HR2_2', 'HR4', 'HR5', 'PERSONAL3', 'MEDIA10', 'MEDIA12',
       'age', 'DEMOGRAPHIC5_46825', 'DEMOGRAPHIC5_81650',
       ...
       'DEMOGRAPHIC5_12225', 'DEMOGRAPHIC5_31323', 'DEMOGRAPHIC5_90005',
       'DEMOGRAPHIC5_18051', 'DEMOGRAPHIC5_25271', 'DEMOGRAPHIC5_43701',
       'DEMOGRAPHIC5_34475', 'DEMOGRAPHIC5_6152', 'DEMOGRAPHIC5_21740',
       'DEMOGRAPHIC5_40353'],
      dtype='object', length=949)
[[0.61768156 0.6431282  0.5358015  0.53402394]
 [0.61768156 0.6431282  0.5358015  0.53402394]
 [0.61768156 0.6431282  0.5358015  0.53402394]
 ...
 [0.61768156 0.6431282  0.5358015  0.53402394]
 [0.61768156 0.6431282  0.5358015  0.53402394]
 [0.61768156 0.6431282  0.5358015  0.53402394]]
470     2
963     3
53      4
210     2
1027    3
       ..
624     1
728     4
881     1
1130    0
610     1
Name: HR6, Length: 303, dtype: int64
2 2.0
3 2.0
4 2.0
2 2.0
3 2.0
0 2

### Testing myself because why not

HR6: Now thinking about healthcare in the country as a whole, overall, what are your impressions of the affordability of healthcare in this country?
Good 3
MEDIA12: Do you now use e-cigarettes or other electronic Ã¢â‚¬Å“vapingÃ¢â‚¬Â products every day, some days, or not at all? 
Every day 2
DEMOGRAPHIC4: In which state are you currently residing (may be different than your permanent address)?
GA DEMOGRAPHIC4_GA = 1
DEMOGRAPHIC5: Please enter your zip or postal code of your permanent address:
DEMOGRAPHIC5_30060 = 1
IDEOLOGY6: Deeply political- highly engaged/follow politics regularly
3

In [155]:
devin = pd.DataFrame(columns = x_train.columns)
devin["HR6"] = [3]
devin["MEDIA12"] = [2]
devin["IDEOLOGY5"] = [3]
devin["DEMOGRAPHIC4_GA"] = 1
devin["DEMOGRAPHIC5_30060"] = 1
devin = devin.fillna(0)
devin = devin.drop("IDEOLOGY5",axis=1)
devin_pred = model.predict(devin)
pred = np.round(np.sum(devin_pred))
if pred == 0:
    print("Far left")
elif pred == 1:
    print("Moderate left")
elif pred == 2:
    print("Neutral")
elif pred == 3:
    print("Moderate right")
elif pred == 4:
    print("Far right")

Far right


I'd say I'm more moderate right but it's very close

In [146]:
ga = x_train.loc[x_train["DEMOGRAPHIC4_AL"] == 1]
zips = []
for index, values in ga.iteritems():
    code = index[-5:]
    if len(values.unique()) > 1 and code.isnumeric():
        zips.append(int(code))
print(zips)
zips.sort()
zips

[36535, 35206, 36207, 35984, 36420, 36541, 35243, 36856, 60478, 54869, 35603, 36350, 36111]


[35206,
 35243,
 35603,
 35984,
 36111,
 36207,
 36350,
 36420,
 36535,
 36541,
 36856,
 54869,
 60478]

### Testing out HR10_scale
	STIMULUS5	DEMOGRAPHIC4	DEMOGRAPHIC5	age	HR1_10SCALE
stimulus5: STIMULUS5_D	= 1
DEMOGRAPHIC4_GA = 1
DEMOGRAPHIC5_30060 = 1
age: 22

In [34]:
devin = pd.DataFrame(columns = x_train.columns)
devin["age"] = [22]
devin["STIMULUS5_D"] = [1]
devin["DEMOGRAPHIC4_GA"] = [1]
devin["DEMOGRAPHIC5_30060"] = [1]
devin = devin.fillna(0)
#devin = devin.drop("HR1_10SCALE",axis=1)
devin_pred = model.predict(devin)
pred = np.round(np.sum(devin_pred))
print(pred+1)

10.0
